# Aritmética modular

## Ejercicio 1

Implementa el algoritmo extendido de Euclides para el cálculo del máximo común divisor: dados dos enteros $a$ y $b$, encuentra $u, v ∈ \mathbb{Z}$ tales que $au + bv$ es el máximo común divisor de $a$ y $b$.

In [1]:
ext_euclides :: Integral a => a -> a -> [a]
ext_euclides a b = ext_euclides' a b

ext_euclides' :: Integral a => a -> a -> [a]
ext_euclides' a 0 = [a, 1, 0]
ext_euclides' 0 b = [b, 0, 1]
ext_euclides' a b = [d, m, n - (a `div` b) * m]
    where
        [d,n,m] = ext_euclides' b (a `mod` b)

ext_euclides 4864 3458

[38,32,-45]

En el código podemos ver como la función `ext_euclides` recibe como parámetros de entrada dos enteros $a$ y $b$ y devuelve el máximo común divisor, seguidos por $u$ y $v$. 

La función sigue el ejemplo de código del algoritmo _2.107_ de [A. Menezes, P. van Oorschot, and S. Vanstone, Handbook of Applied Cryptography, CRC Press, 1996.](http://cacr.uwaterloo.ca/hac/about/chap2.pdf)

## Ejercicio 2

Usando el ejercicio anterior, escribe una función que calcule $a^{-1} \bmod b$ para cualesquiera $a, b$ enteros que sean primos relativos.

In [2]:
inverse :: Integral a => a -> a -> a
inverse a b = ext_euclides a b !! 1 

inverse 2 5

-2

A partir del código del ejercicio 1, en caso de que exista inversa en $\mathbb{Z}_n$, obtendremos lo siguiente: $$d = au + bv$$ En caso de que $a$ tenga inversa en $\mathbb{Z}_n$, tendremos que $\text{mcd}(a,n) = 1$. Por tanto, por la identidad de Bezout, tenemos que existen $u$ y $v$ (coeficientes de Bezout) tal que: $$1 = ua + vn$$

Por tanto, si estamos en el espacio $\mathbb{Z}_n$, tenemos que $$ \begin{matrix}1 = ua + vn & =& ua + 0 \\ & \Rightarrow & ua & a \in \mathcal{U}(\mathbb{Z}_n)\\  u & = & a^{-1} \end{matrix}$$

Para devolver el inverso correcto, devolveremos $u \bmod n$.

## Ejercicio 3

Escribe una función que calcule $a^b \bmod n$ para cualesquiera $a, b\text{ y } n$. La implementación debe tener en cuenta la representación binaria de $b$.

In [3]:
big_pow :: Integral a => a -> a -> a -> a
big_pow _ 0 _ = 1
big_pow a b n = pow a b 1 n

pow :: Integral a => a -> a -> a -> a -> a
pow _ 0 p _ = p
pow a b p n 
        | b `mod` 2 == 1 = pow (a*a `mod` n) (b `div` 2) ((p * a) `mod` n) n
        | otherwise      = pow (a*a `mod` n) (b `div` 2) p n

        
big_pow 156187561565735418 43498489489156978415674 23

16

Para calcular $a^b \bmod n$, podemos tomar como base que, el exponente $b$ puede escribirse en binario como $b = b_0b_1\ldots b_k$ tal que $b_i = 0\;|\;1$. A partir de esto, podemos definir $b$ como $b = \sum_{i=0}^k b_i\cdot2^i$.

Entonces, la expresión $a^b$ se puede expresar como: $$a^b = a^{\sum_{i=0}^k b_i\cdot2^i} = \prod a^{b_i2^i} = \prod \left(a ^{2^i}\right)^{b_i}$$

Con esto podemos ver que el valor del producto se incrementará cuando el valor de $b_i = 1$, elevándose el valor del producto al cuadrado.

Una forma de realizar este cálculo, es la que aparece en el código, y es ir realizando las operaciones $b_i = b \bmod 2; \quad b = \lfloor b | 2 \rfloor$, e ir siempre incrementando el valor de $a$ como $a = a^2 \bmod n$. El valor del producto, denotado como $p$, se incrementará cuando el valor de $b = 1$, tal que $p = (p \cdot a)\bmod n$.

## Ejercicio 4

Dado un entero $p$, escribe una función para determinar si $p$ es probablemente primo usando el método de Miller-Rabin.

In [4]:
import System.Random
import System.IO.Unsafe
import Data.List

-- Descompone un número p tal que p = 2^s * u
bifactor :: Integral a => a -> [a]
bifactor num = bifactor' num 0

bifactor' :: Integral a => a -> a -> [a]
bifactor' 0 s = [s, 1]
bifactor' a0 s 
    | a0 `mod` 2 == 0 = bifactor' (a0 `div` 2) (s + 1)  -- si a | b => s+=1 y comprobamos con a / 2
    | otherwise       = [s, a0]                         -- en caso contario, devolvemos s y u


miller_rabin :: (Integral a, Random a) => a -> Bool
miller_rabin p 
    | p == 2 || p == 3                   = True  -- primos menores que 5
    | p == 4 || p < 2 || p `mod` 2 == 0  = False -- es 4 o menor que 4 
    | otherwise                          = test_mr p l
    where
        s_u = bifactor (p - 1)  -- Descomponemos p - 1 = 2^s * u --> [s, u]
        a = unsafePerformIO $ randomRIO (2, p - 2) -- obtenemos una semilla aleatoria para el test
        l = map (\x -> big_pow a (2^x * s_u !! 1) p) [0..s_u !! 0] -- y construimos la lista

        test_mr :: Integral a => a -> [a] -> Bool
        test_mr p l
            -- Primer elemento de la lista es 1 o -1
            | (head l) == 1 || (head l) == (p - 1)                 = True  -- Probablemente primo
            -- Si -1 está en la lista y no es el último elemento
            | (p - 1) `elem` l && (last l /= p - 1)                = True  -- Probablemente primo
            -- Ninguna de las potencias es igual a 1
            | not (1 `elem` l)                                     = False -- No es primo
            -- Si aparece un 1 en la lista no precedido de un -1
            | 1 `elem` l && (last $ takeWhile (/= 1) l) /= (p - 1) = False -- No es primo
            -- En otro caso
            | otherwise                                            = False -- No es primo

miller_rabin_test :: (Integral a, Random a) => a -> Int -> Bool
-- Realiza un and con las n salidas del test de Miller-Rabin
miller_rabin_test p n = and $ replicate n (miller_rabin p)  

miller_rabin_test 123456789101119 10
miller_rabin_test 127973 10
miller_rabin_test 127972 10
miller_rabin_test 123456789101119 10
miller_rabin_test 28564333765949 10

True

True

False

True

True

El test de Miller-Rabin lo realiza la función `miller_rabin`. Esta función se encarga de dado un "primo" $p$, se encarga de descomponer $p-1$ en $p-1 = 2^s\cdot u$. 

Con esto compone la lista $l = [a^{2^0u}, a^{2^0u}, \ldots, a^{2^su}]$, siendo $a \in_R [2, \ldots, p - 2]$ y pasamos a comprobar las condiciones de ser ___probablemente primo___ o ___no primo___:
* Si el primer elemento de $l$ es 1 o -1, $p$ será probablemente primo.
* Si no aparece 1 en $l$, es no primo, ya que no supera el test de Fermat.
* Si aparece un 1 no precedido de -1, es no primo, ya que existe un raíz cuadrada de 1 que no es ni 1 ni -1.
* Si aparece un -1 en $l$, es probablemente primo, ya que el siguiente elemento en la lista será 1.

## Ejercicio 5

Implementa el algoritmo paso enano-paso gigante para el cálculo de algoritmos discretos en $\mathbb{Z}_p$.

In [5]:
-- Función para devolver el índice como un entero
-- en vez de un Maybe Int con elemIndex
indexOf :: Integral a => a -> [a] -> a
indexOf y xs = index y xs 0
    
index :: Integral a => a -> [a] -> a -> a
index _ [] n            = -1 
index y (x:xs) n
            | y /= x    = index y xs n + 1 
            | otherwise = n

baby_pass_giant_pass :: (Integral a, Random a) => a -> a -> a -> [a]
baby_pass_giant_pass _ 1 _ = [0] -- Si b == 1, devolvemos un 0
baby_pass_giant_pass a b p 
    -- Si es primo, devolvemos los ks que cumplen a^k = b en Z_p
    | (miller_rabin_test p 5) = ks
    | otherwise               = []
    where
        s = ceiling $ sqrt $ fromIntegral (p-1)
        -- Calculamos el paso gigante => L
        big_pass = map (\x -> big_pow a (x * s) p) [1..s] 
        -- Calculamos el paso pequeño => l
        low_pass = map (\x -> (b * a^x) `mod` p ) [0..s - 1] 
        -- Realizamos la intersección de L y l y 
        -- calculamos para la lista resultante
        -- para cada k en ks => k = cs - r
        ks = map (\x -> ((indexOf x big_pass) + 1) * s - 
            (indexOf x low_pass)) (intersect big_pass low_pass)
        
baby_pass_giant_pass 6 32 41

[10]

La función realiza el cálculo del algoritmo paso enano-paso gigante, comprobando primero que $p$ es primo. En caso de no serlo, devuelve un valor nulo. En caso de que sea primo, comprueba si $b = 1$ o no. Si lo es, devuelve $k = 0$ ya que $a^0 = 1$. En caso contrario pasamos a realizar el algoritmo paso enano-paso gigante:
* __Paso gigante__: calcularemos la lista $L$ como $L = [a^s, a^{2s}, \ldots, a^{ss}]$, donde en cada paso, se multiplica el valor anterior por $a^s$.
* __Paso enano__: calcularemos la lista $l$ como $l = [b, ba, \ldots, ba^{s-1}]$, donde en cada paso, multiplicamos el valor anterior por $a$.

Si $L \cap l \neq \emptyset$, existe al menos un $k$ tal que $a^k = b$ con $a,b \in \mathbb{Z}_p$. Además, tenemos que $$a^{cs}\in L \quad = \quad ba^r \in l$$ por lo tanto $k = cs - r$.

La función `map` se encarga de calcular el exponente $k$ para todos los elementos que aparecen de la intersección de $L$ (*big_pass*) y $l$ (*low\_pass*), y nos devuelve la lista con todos los $k$ calculados. En caso de que la intersección sea vacía, devuelve una lista vacía.

## Ejercicio 6

Sea $n = pq$, con $p$ y $q$ enteros primos positivos.
* Escribe una función que, dado un entero $a$ y un primo $p$ con $\left(\frac{a}{p}\right) = 1$, devuelva $r$ tal que $r^2 \equiv a \bmod p$; primero te hará falta implentar el símbolo de Jacobi.
* Sea $a$ un entero que es residuo cuadrático módulo $p$ y $q$. Usa el teorema chino de los restos para calcular todas las raíces cuadradas de $a$ módulo $p$ y $q$.

In [6]:
jacobi :: Integral a => a -> a -> a
jacobi a p 
    -- 1. (a / p) = (a mod p / p)
    | a > p      = jacobi (a `mod` p) p 
    -- 2. (ab / p) = (a / p) * (b / p)
    -- Esta descomposición se hace descomponiendo n en (2^u)*s
    -- y usamos la regla de (2 / p) = (-1)^((p^2 - 1) / 8)
    | u > 0      = ((-1) ^ (((p^2) - 1) `div` 8))^u * jacobi' s p
    | otherwise  = jacobi' s p
        where
            u_s = bifactor a
            u   = head u_s
            s   = last u_s
            jacobi' :: Integral a => a -> a -> a
            jacobi' a p 
                | a == 0      = 0  -- Si a = 0, devolvemos 0
                | a == 1      = 1  -- Si a = 1, devolvemos 1
                | a == -1     = (-1)^((p - 1) `div` 2)   -- Si a = -1, devolvemos (-1)^((p-1)/2)
                | a `mod` 2 /= 0 && p `mod` 2 /= 0 = jacobi p a -- Si ambos son impares (p / q) => (q / p)
                | otherwise                        = jacobi a p -- En caso contrario, seguimos con (p / q)
                

jacobi 5 1009

1

En esta función implementamos el símbolo de Jacobi. Esta implementación, aplica las siguientes reglas, prácticamente en el mismo orden de aparición:
1. $\left(\frac{a}{p}\right) = \left(\frac{a \bmod p}{p}\right)$
2. $\left(\frac{ab}{p}\right) = \left(\frac{a}{p}\right)\left(\frac{b}{p}\right)$. Esta regla, en particular, lo que hacemos es descomponer el número $n$ en $n = 2^su$, por tanto, esto pasa a ser $$\left(\frac{n}{p}\right) = \left(\frac{2^u}{p}\right)\left(\frac{s}{p}\right)$$
3. $\left(\frac{2}{p}\right) = (-1)^\frac{p^2 - 1}{8}$
4. $\left(\frac{1}{p}\right) = 1$
5. $\left(\frac{1}{p}\right) = (-1)^\frac{p-1}{2}$
6. $\left(\frac{q}{p}\right)$, si $p$ y $q$ son ambos impares, hacemos lo siguiente: $$ \left(\frac{q}{p}\right) = (-1)^\frac{(p - 1)(q - 1)}{4}\left(\frac{p}{q}\right) =  \begin{cases}
     - \left(\frac{p}{q}\right) & \quad \text{si } p \equiv q \equiv 3 \bmod 4 \\
    \left(\frac{p}{q}\right) & \quad \text{en caso contrario}\\
  \end{cases}$$
  
Esta función calcula el símbolo de Jacobi de forma recursiva, pero no es una recursividad tan fuerte ya que, al descomponer $n$ en $n = 2^su$, aplicamos la regla 3, calculando el símbolo de la primera parte, y ya nos centramos en calcular el símbolo de $\left(\frac{u}{p}\right)$. A continuación, vamos a usar esto para calcular la raíz modular.

In [7]:
-- Devuelve las dos raíces de un número a en Z_p
sqrts_mod :: Integral b => b -> b -> [b]
sqrts_mod a p = sort [sqr, p - sqr] -- Devolvemos la lista ordenada
    where
        sqr = sqrt_mod a p

-- Calcula una de las raíces modulares
sqrt_mod :: (Integral b) => b -> b -> b
sqrt_mod a p
    | jacobi a p /= 1 = error "No existen raíces para a mod p" 
    | u == 1          = big_pow a ((p + 1) `div` 4) p -- Si U = 1, sqrt(b)_p = a^((p + 1) / 4) en Z_p
    | otherwise       = search_residual a u s n p -- Buscamos el residuo de forma "iterativa"
        where 
            u_s = bifactor (p - 1)  -- Descomponemos p-1 en 2^u s
            u = head u_s
            s = last u_s
            -- Y obtenemos (n / p) == -1
            n =  1 + (last $ takeWhile (\x -> (jacobi x p) == 1) [2..p - 1])

-- Busca el residuo de a mod p 
search_residual :: (Integral b) => b -> b -> b -> b -> b -> b
search_residual a u s n p = res r b j inv_a u p
    where
        -- Inicializamos el algoritmo
        r     = big_pow a ((s + 1) `div` 2) p
        b     = big_pow n s p
        j     = 0
        inv_a = inverse a p
        -- Buscamos los residuos
        res :: (Integral b) => b -> b -> b -> b -> b -> b -> b
        res r b j inv_a u p 
            -- Caso base
            | j > (u - 2) = r 
            -- Si (a^-1 * r^2)^(2 ^ (u - 2 - j)) == -1 => actualizamos el valor de r 
            -- y seguimos "iterando"
            | (big_pow (inv_a * r^2) (2^(u - 2 -j)) p) == (p - 1) = res ((r * b) `mod` p) (b^2) (j+1) inv_a u p
            -- Si no se da el caso, seguimos iterando
            | otherwise = res r (b^2) (j+1) inv_a u p
            
sqrts_mod 64 1009

[8,1001]

Una vez que tenemos las raíces en un número $a$ en un cuerpo $\mathbb{Z}_p$, podemos usar el ___Teorema Chino de los Restos___ para hallar las raíces de un número $a$ en $\mathbb{Z}_n$, donde $n = p\cdot q$. Para ello, usaremos las siguientes funciones:

In [8]:
-- Normaliza la congruencia para que sea de la forma
-- (x r m) => (1 e f)
norm :: (Integral a) => (a, a, a) -> (a, a, a)
norm (x, y, m)
    | y `mod` d /= 0 = error "Error: congruencia sin solución"
    | otherwise      = (1, e, f)
    where
        euc = ext_euclides x m
        d = head euc
        s = euc !! 1
        h = y `div` d
        f = m `div` d
        e = (h * s) `mod` f
        
-- Implementa el teorema chino de los restos 
chinese_remainder :: (Integral a) => (a, a, a) -> (a, a, a) -> a -> a
chinese_remainder (x, a, p) (x', b, q) n = sol
    where
        (x1, a', p') = norm (x, a, p)  -- Normalizamos las congruencias
        (x2, b', q') = norm (x', b, q) -- por si fuera necesario
        inv_p = inverse p q            -- Calculamos el inverso de p mod q
        aux = (b' - a') * inv_p        
        sol = (a' + p * aux) `mod` n   -- y obtenemos la solución de la congruencia


sqrts_mod_n :: (Integral b) => b -> b -> b -> [b]
sqrts_mod_n a p q = sort [root1, root1', root2, root2']
    where
        sqrts_p = sqrts_mod a p
        sqrts_q = sqrts_mod a q
        n       = p * q
        root1   = chinese_remainder (1, (head sqrts_p), p) (1, (head sqrts_q), q) n
        root2   = chinese_remainder (1, (last sqrts_p), p) (1, (head sqrts_q), q) n
        root1'  = n - root1
        root2'  = n - root2
        
sqrts_mod_n 8 17 41

[294,335,362,403]

El cálculo de la raíz cuadrada usando el Teorema Chino de los Restos se realiza calculando las raíces del número $a$ módulo $p$ y $q$ siendo ambos primos. Una vez calculados tendremos: $$ \sqrt{a}_{\mathbb{Z}_p} = \{r_1, r_2\} \qquad \sqrt{a}_{\mathbb{Z}_q} = \{r_3, r_4\} $$

Con esto, podemos construir un sistema de congruencias para hallar la raíz de $a$ en $\mathbb{Z}_n$:$$\begin{matrix}
x&\equiv r_1 \bmod p\\
x&\equiv r_3 \bmod q\\
\end{matrix}$$

Para solucionarlo seguimos los siguientes pasos:$$ x = r_1 + p\cdot t$$ Sustituimos $x$ por la expresión obtenida en la segunda congruencia: $$
\begin{matrix}
r_1 + p\cdot t & \equiv & r_3 \bmod q\\
p\cdot t & \equiv &(r_3 - r_1) \bmod q\\
t & \equiv & (r_3 - r_1)\cdot p^{-1} \bmod q\\
\end{matrix}$$
Por tanto, $t$ queda como $$t = (r_3 - r_1)\cdot p^{-1} + q\cdot s$$ Si sustituimos $t$ en la primera expresión, obtenemos $$\begin{matrix}
x &=& r_1 + p\cdot ((r_3 - r_1)\cdot p^{-1} + q\cdot s)\\
& = & r_1 + p\cdot ((r_3 - r_1)\cdot p^{-1}) + ({p\cdot q\cdot s)}\\
& = & r_1 + p\cdot ((r_3 - r_1)\cdot p^{-1})
\end{matrix}$$

Con esto sólo obtenemos una de las raíces. La otra puede calcularse haciendo el sistema de congruencias con $r_2$ y $r_4$, pero no es necesario ya que esta raíz es la complementaria de la anterior en $\mathbb{Z}_n$. Las otras dos se calculan planteando un sistema de congruencias con $r_1$ y $r_4$ y calculando el complementario.

## Ejercicio 7

* Implementa el Método de Fermat para factorización de enteros.
* Implementa el algoritmo de factorización $\rho$ de Pollard

In [9]:
fermat :: Integral a => a -> [a]
fermat n = fermat' x n
    where
        --                       _         _
        -- valor inicial de x = |  sqrt(n)  |
        x       = ceiling $ sqrt (fromIntegral n)

fermat' :: Integral a => a -> a -> [a]
fermat' x n
    -- Si no encontramos solución, devolvemos la lista vacía
    | x >= n    = []
    -- SI la raíz es entera, devolvemos la factorización
    | cond      = [x - high_sqrt, x + high_sqrt]
    -- En caso contrario, seguimos buscando
    | otherwise = fermat' x' n 
        where
            sqrt_val  = sqrt (fromIntegral (x^2 - n))
            low_sqrt  = floor $ sqrt_val
            high_sqrt = ceiling $ sqrt_val
            cond      = high_sqrt == low_sqrt
            x' = x + 1
            
fermat 6352351
fermat 5959

[2389,2659]

[59,101]

El método de Fermat recibe un número $n$ e intenta factorizarlo de manera que, tomando un $x$ y un $y$ que cumplan $$x^2 - y^2 = n$$ Esto se debe a que si se cumple la igualdad anterior, tenemos que $$x^2 - y^2 = \underbrace{(x-y)(x+y)}_{factores} = n$$ De ser así, $x^2 - n = y^2$ es un cuadrado.

Por tanto, para calcular los factores, inicializa el valor de $x = \lceil \sqrt{n} \rceil$ y va comprobando si $x^2 - n$ es un cuadrado, usando el símbolo de Jacobi. En caso de que no lo sea, comprobará con $x + 1$, así hasta que encuentre los factores de $n$.

In [10]:
pollard :: Integral a => a -> a -> a -> a -> (a -> a) -> a
pollard n x y d f
    | d == 1    = pollard n x' y' d' f
    | d == n    = error "Error: no se encuentra descomposición para n"
    | otherwise = d
    where
        x' = f x
        y' = f (f y)
        d' = head $ ext_euclides (abs (x' - y')) n


rho_pollard :: Integral a => a -> a -> [a]
rho_pollard n c 
    | c == 0 || c == -2 = error "Error: c tiene que ser distinto de 0 y -2"
    | otherwise         = [factor, n `div` factor]
    where
        f = \x -> (x^2 + c) `mod` n
        --       pollard n x y d f
        factor = pollard n 2 2 1 f
        
rho_pollard 6352351 1
rho_pollard 5959 1

[2389,2659]

[59,101]

El algoritmo $\rho$ de Pollard para la factorización de enteros, va generando dos cadenas de números de forma pseudo-aleatoria, con el fin de descomponer el número $n$ en factores. Estas cadenas de números se generan en base a una función $f$ definida como $$f(x) = x^2 + c \bmod n$$ donde $n$ es el número a descomponer, $c \in \mathbb{Z}\backslash\{0, -2\}$ y $x$ será un número "aleatorio" que al inicio del algoritmo tendrá el valor 2, al igual que $y$.

La función tendrá un bucle que ciclará hasta que $d \neq 1$, siendo $d = mcd(|x - y|, n)$. En el bucle, se actualizarán los valores de $x$ e $y$ como $x = f(x)$ e $y = f(f(y))$.

En el momento en el que se cumpla la condición de parada, si $d = n$, se devolverá un error, y será necesario probar con otro valor de $c$. En caso contrario, se devolverá el factor encontrado y se calculará el otro como la división de $n$ entre el factor encontrado.